# 1.0 Imports

In [ ]:
from tqdm import tqdm
from openai import OpenAI
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math
import os
from dotenv import load_dotenv

# 2.0 Setting Model

In [ ]:
# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

# 3.0 Helper Functions

In [ ]:
def classify_product(product_name, categories, known_examples):
    examples = "\n".join([f"Product: {k}, Category: {v}" for k, v in known_examples.items()])
    prompt = f"""Here are some examples of product classifications:

    {examples}

    Now, classify the following product into a category.
    Follow the step by step below:
    1. Use ONLY the categories {', '.join(categories)} to classify.
    2. Just provide the classification, without any explanation.
    3. Use only the classification provided in {', '.join(categories)} to classification.
    4. Don't create new categories.

    Product: {product_name}
    Possible categories: {', '.join(categories)}
    Category:"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4096,
        n=1,
        stop=None,
        temperature=0.0,
    )

    return response.choices[0].message.content.strip()

# Function to classify products in a DataFrame
def classify_dataframe_products(df, product_column, batch_size=600):
    predictions = []
    for i in tqdm(range(0, len(df), batch_size), desc="Classifying products"):
        batch = df[product_column].iloc[i:i+batch_size]
        batch_predictions = [classify_product(product, categories, known_examples) for product in batch]
        predictions.extend(batch_predictions)
    return predictions

# Function to calculate similarity using cosine similarity
def calculate_similarity(row, vectorizer):
    vectors = vectorizer.transform([row['categoryName'], row['categoria predita']])
    return cosine_similarity(vectors[0:1], vectors[1:2])[0][0]


# 4.0 Load Dataset

In [ ]:
df_raw = pd.read_csv("amz_br_total_products_data_processed.csv")

# 5.0 Cleaning NaN values

In [ ]:
df_raw = df_raw.fillna('sem_informacoes')
df_raw.isna().sum()

asin                 0
title                0
imgUrl               0
productURL           0
stars                0
reviews              0
price                0
listPrice            0
categoryName         0
isBestSeller         0
boughtInLastMonth    0
dtype: int64

# 6.0 Pic a 50 samples from dataset

In [ ]:
df = df_raw.loc[:, ["title", "categoryName"]].sample(50)
df

,title,categoryName
1331165,"Mini Piscina Redonda Inflável Para Bebês, Pisc...","Piscinas, Ofurôs e Materiais"
1297635,Toyvian 3Pcs Copo Portátil Garrafas De Água Es...,Alimentação de Bebês e Crianças Pequenas
584938,9-13 [Disco de Vinil],CD e Vinil
539221,Alto-falante para carro – Par de alto-falantes...,Eletrônicos e Tecnologia Automotivos
7395,Deo Colônia Dolce Pet Cereja e Avelã para Cães...,Pet Shop
173172,"Global Translator, Smart Translator Microfone ...",Tablets
851487,Leapiture adaptador de diagnóstico obd2 obd2 c...,"PlayStation 4, Jogos, Consoles e Acessórios"
813633,Ipetboom Brinquedo Para Mastigar Papagaio Brin...,Produtos para Aves e Pássaros
775402,Pressão Conjuntos Ou Alfinetes De Fivelas De R...,"Roupas, Calçados e Joias"
813125,Parliky Brinquedo Para Mastigar Para Pássaros ...,Produtos para Aves e Pássaros


# 7.0 Implementing the IA Classification Model

In [ ]:
# Load your dataset with known categories
df_known = df.head(40)

# Load your dataset to classify
df_compare = df.tail(10)

# Create a dictionary from df_known
known_examples = dict(zip(df_known['title'], df_known['categoryName']))

# Get unique categories
categories = df['categoryName'].unique().tolist()

# Defying test data
df_new_products = df_compare
df_new_products = df_new_products.drop(('categoryName'), axis=1)

# Classify the new products
df_new_products['categoria predita'] = classify_dataframe_products(df_new_products, 'title')

# Optional: If you want to merge this with any existing data
df_final = pd.merge(df_compare.loc[:, ["title", "categoryName"]], df_new_products.loc[:, ["categoria predita"]], left_index=True, right_index=True, how='inner')
df_final

Classifying products: 100%|██████████| 1/1 [00:44<00:00, 44.15s/it]


,title,categoryName,categoria predita
618777,Botas de caminhada para meninos com cintura al...,Meninos,Meninos
248689,Hosioe TTC-450 Engraver Engraving and Cutting ...,Ferramentas Elétricas,Ferramentas Elétricas
359493,Tapete Yoga Infantil PVC Estampado para Crianç...,Equipamento para Exercícios e Academia,Equipamento para Exercícios e Academia
1140977,Relógios Digitais Para Mulheres Relógio De Cri...,Relógios,Relógios
1231986,Rede Para Banheira Rosa Baby Style,"Banho, Higiene e Troca de Fraldas do Bebê","Banho, Higiene e Troca de Fraldas do Bebê"
328668,Angoily Braço de chuveiro com extensão de braç...,Instalações de Cozinha e Banheiro,Instalações de Cozinha e Banheiro
997337,Generic 2 Peças Organizador De Acessórios Elet...,Acessórios para Viagem,Acessórios para Viagem
940902,ibasenice 6 Pçs Suporte De Soquete Ferramenta ...,Cuidados Automotivos,Organizadores de Ferramentas
244751,Cancanle 10 Peças de Vedação de Óleo para Moto...,Cortadores e Ferramentas Elétricas para Ambien...,Cortadores e Ferramentas Elétricas para Ambien...
875662,VILLFUL 2 Peças De Material De Papelaria Canet...,Materiais de Escrita e Correção,Materiais de Escrita e Correção


# 8.0 Validation metrics

In [ ]:
# Create a TF-IDF Vectorizer
combined_text = pd.concat([df_final['categoryName'], df_final['categoria predita']])
vectorizer = TfidfVectorizer().fit(combined_text)

# Calculate similarity
df_final['similaridade'] = df_final.apply(lambda row: calculate_similarity(row, vectorizer), axis=1)

# Calculate and print average similarity
average_similarity = df_final['similaridade'].mean()
print(f"\nAverage Similarity: {average_similarity:.2f}")

# Optional: You can also add a column to flag if the prediction is correct (assuming an exact match is required)
df_final['validacao'] = df_final['categoryName'] == df_final['categoria predita']

# Print accuracy
accuracy = df_final['validacao'].mean()
print(f"Accuracy: {accuracy:.2f}")

df_final


Average Similarity: 0.90
Accuracy: 0.90


,title,categoryName,categoria predita,similaridade,validacao
618777,Botas de caminhada para meninos com cintura al...,Meninos,Meninos,1.0,True
248689,Hosioe TTC-450 Engraver Engraving and Cutting ...,Ferramentas Elétricas,Ferramentas Elétricas,1.0,True
359493,Tapete Yoga Infantil PVC Estampado para Crianç...,Equipamento para Exercícios e Academia,Equipamento para Exercícios e Academia,1.0,True
1140977,Relógios Digitais Para Mulheres Relógio De Cri...,Relógios,Relógios,1.0,True
1231986,Rede Para Banheira Rosa Baby Style,"Banho, Higiene e Troca de Fraldas do Bebê","Banho, Higiene e Troca de Fraldas do Bebê",1.0,True
328668,Angoily Braço de chuveiro com extensão de braç...,Instalações de Cozinha e Banheiro,Instalações de Cozinha e Banheiro,1.0,True
997337,Generic 2 Peças Organizador De Acessórios Elet...,Acessórios para Viagem,Acessórios para Viagem,1.0,True
940902,ibasenice 6 Pçs Suporte De Soquete Ferramenta ...,Cuidados Automotivos,Organizadores de Ferramentas,0.0,False
244751,Cancanle 10 Peças de Vedação de Óleo para Moto...,Cortadores e Ferramentas Elétricas para Ambien...,Cortadores e Ferramentas Elétricas para Ambien...,1.0,True
875662,VILLFUL 2 Peças De Material De Papelaria Canet...,Materiais de Escrita e Correção,Materiais de Escrita e Correção,1.0,True
